# Query Drivers by Page

Use this notebook to understand which queries are driving each page and how concentrated performance is across a few terms.

In [ ]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

import pandas as pd
import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import build_date_params, default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()
window = default_query_window(config.DEFAULT_DAYS_BACK)
PAGE_PATH = "/"  # update this page path to inspect a specific page

In [11]:
query = f"""
SELECT
  report_date,
  page_path,
  query,
  SUM(clicks) AS clicks,
  SUM(impressions) AS impressions,
  SAFE_DIVIDE(SUM(clicks), NULLIF(SUM(impressions), 0)) AS ctr,
  SAFE_DIVIDE(SUM(avg_position * impressions), NULLIF(SUM(impressions), 0)) AS avg_position
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.curated_search_query_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
  AND page_path = @page_path
GROUP BY report_date, page_path, query
ORDER BY report_date DESC, clicks DESC
"""

from google.cloud import bigquery

params = build_date_params(window) + [
    bigquery.ScalarQueryParameter("page_path", "STRING", PAGE_PATH),
]

df_queries = run_query(client, query, params=params)
df_queries.head(20)

/home/aido/projects/lla-data/.venv/lib/python3.14/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,report_date,page_path,query,clicks,impressions,ctr,avg_position
0,2026-02-20,/get-help/national-services/national-alcohol-o...,NaN,261,18511,0.014100,1.960564
1,2026-02-20,/get-help/national-services/national-alcohol-o...,how long do dexies stay in your system,4,20,0.200000,1.100000
2,2026-02-20,/get-help/national-services/national-alcohol-o...,alcoholics anonymous 24 hour phone number,2,2,1.000000,2.500000
3,2026-02-20,/get-help/national-services/national-alcohol-o...,heroin,2,58,0.034483,1.775862
4,2026-02-20,/get-help/national-services/national-alcohol-o...,mdma overdose,2,5,0.400000,1.000000
5,2026-02-20,/get-help/national-services/national-alcohol-o...,how long does ecstasy take to wear off,2,2,1.000000,1.500000
6,2026-02-20,/get-help/national-services/national-alcohol-o...,directline aod,2,2,1.000000,2.000000
7,2026-02-20,/get-help/national-services/national-alcohol-o...,aa,2,387,0.005168,2.674419
8,2026-02-20,/get-help/national-services/national-alcohol-o...,medication to stop drinking,2,6,0.333333,3.000000
9,2026-02-20,/get-help/national-services/national-alcohol-o...,aa zoom meetings,2,12,0.166667,1.000000


In [12]:
top_queries = (
    df_queries.groupby("query", as_index=False)[["clicks", "impressions"]]
    .sum()
    .sort_values("clicks", ascending=False)
    .head(20)
)

total_clicks = float(top_queries["clicks"].sum()) or 1.0
top_queries["click_share"] = top_queries["clicks"] / total_clicks

fig = px.bar(
    top_queries.sort_values("clicks", ascending=True),
    x="clicks",
    y="query",
    orientation="h",
    template="lifeline",
    title=f"Top Query Drivers for {PAGE_PATH}",
)
fig.update_layout(height=700, margin={"l": 320})
lifeline_theme.add_lifeline_logo(fig)
fig.show()

top_queries.head(10)

,query,clicks,impressions,click_share
257,aa,5,599,0.096154
1310,alcoholics anonymous,4,75,0.076923
3736,drug helpline,4,13,0.076923
5304,how long do dexies stay in your system,4,25,0.076923
3608,drug and alcohol counselling,3,47,0.057692
4909,heroin,3,131,0.057692
7711,mdma overdose,3,10,0.057692
8442,narcotics anonymous,2,46,0.038462
5431,how long does ecstasy take to wear off,2,2,0.038462
8464,national alcohol and other drug hotline,2,13,0.038462
